In [6]:
import requests
import scipy
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import numpy
from functools import reduce
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation
import re
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from sklearn.cluster import MiniBatchKMeans
import unicodedata
stop = set(stopwords.words('english'))
import os
with open("config.json") as json_file: 
    config = json.load(json_file)

#break the descriptions into sentences and then break the sentences into tokens
#remove punctuation and stop words
#lowercase the tokens
def tokenizer(text):
    try:
        tokens_ = [word_tokenize(sent) for sent in sent_tokenize(text)]
        
        tokens = []
        for token_by_sent in tokens_:
            tokens += token_by_sent

        tokens = list(filter(lambda t: t.lower() not in stop, tokens))
        tokens = list(filter(lambda t: t not in punctuation, tokens))
        tokens = list(filter(lambda t: t not in [u"'s", u"n't", u"...", u"''", u'``', 
                                            u'\u2014', u'\u2026', u'\u2013'], tokens))
        filtered_tokens = []
        for token in tokens:
            if re.search('[a-zA-Z]', token):
                filtered_tokens.append(token)

        filtered_tokens = list(map(lambda token: token.lower(), filtered_tokens))

        return filtered_tokens
    except Error as e:
        print(e)
#Set the paths
path = os.path.abspath(os.path.dirname("./CORPUS/"))
outPath = os.path.abspath(os.path.dirname("./CLUSTER/"))
directory = path
#Data for clustering
datas = []
#For each file 
for file in os.listdir(directory):
    #concat all of the files together
    data = pd.read_json(directory+"/"+file)
    datas.append(data);

    result = pd.concat(datas)
    result.head
    vectorizer = TfidfVectorizer(min_df=10, max_features=10000, tokenizer=tokenizer, ngram_range=(1, 2))
    vz=vectorizer.fit_transform(list(data['title']))
    num_clusters = 30
    kmeans_model = MiniBatchKMeans(n_clusters=num_clusters, init='k-means++', n_init=1, 
                                   init_size=1000, batch_size=1000, verbose=False, max_iter=2000)

    #Fit the models
    kmeans = kmeans_model.fit(vz)
    kmeans_clusters = kmeans.predict(vz)
    kmeans_distances = kmeans.transform(vz)


    outfile = open(outPath+'/'+os.path.splitext(file)[0]+'.txt', 'w')
    for (i, data.count),category in zip(enumerate(data.title),data['title']):
        s= "Cluster " + str(kmeans_clusters[i]) + ": " + data.count + "(distance: " + str(kmeans_distances[i][kmeans_clusters[i]])+ ")"
        s=unicodedata.normalize('NFKD',s).encode('ascii','ignore')
        print >>outfile,s
        s='category: ',category;
        s=unicode(s)
        #Deal with unicode chars
        s=unicodedata.normalize('NFKD',s).encode('ascii','ignore')
        
        print >>outfile,s
        print >>outfile,'---'
    outfile.close()